In [10]:
import pandas as pd

# make sure the aps module is in the pythonpath
import sys
from pathlib import Path

APS_dir = str(Path.cwd().parents[1])
if APS_dir not in sys.path:
    sys.path.append(APS_dir)
import aps.aps_io.get_forecasts as gf

# Get API data
Request necessary data from https://api01.nve.no/hydrology/forecast/avalanche/v4.0.0/api

Insert relevant data into SQLite database.

General:
- date
- region
- region-ID
- danger level

Avalanche problem:
- avalanche problem name
- avalanche problem order
- trigger
- probability
- elevation

Mountain weather:
- avg. precipitation
- max precipitation (most exposed area)
- temperature (at diff. levels)
- wind speed
- wind direction

In [11]:
warnings, url = gf.get_warnings_as_json(3024, "2018-01-01", "2018-02-01", lang_key=1, simple=False, recursive_count=5)
print(url)

df = pd.DataFrame(warnings)
df.head(5)

https://api01.nve.no/hydrology/forecast/avalanche/v4.0.0/api/AvalancheWarningByRegion/Detail/3024/1/2018-01-01/2018-02-01


,Author,AvalancheDanger,AvalancheProblems,CountyList,CurrentWeaklayers,DangerLevel,DangerLevelName,EmergencyWarning,LangKey,LatestAvalancheActivity,...,RegionId,RegionName,RegionTypeId,RegionTypeName,SnowSurface,UtmEast,UtmNorth,UtmZone,ValidFrom,ValidTo
0,jostein@nve,Små nebørsmengder vil ikkje føre til noko vese...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",28.12 vert det observert overflaterim. Rimet f...,2,2 Moderat,Ikke gitt,1,None,...,3024,Sunnmøre,10,A,I høgfjellet er snøoverflata stort sett hard o...,62473,6916553,33,2018-01-01T00:00:00,2018-01-01T23:59:59
1,jostein@nve,Små nebørsmengder vil ikkje føre til noko vese...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",28.12 vert det observert overflaterim. Rimet f...,2,2 Moderat,Ikke gitt,1,30.12 gjekk det eit skred ved Lafjellet.,...,3024,Sunnmøre,10,A,I høgfjellet er snøoverflata stort sett hard o...,62473,6916553,33,2018-01-02T00:00:00,2018-01-02T23:59:59
2,Halvor@NVE,Perioder med litt snø og vind de siste dagene ...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",Det er observert overflaterim i regionen.,2,2 Moderat,Ikke gitt,1,Det er ikke registrert skredaktivitet de siste...,...,3024,Sunnmøre,10,A,Det har kommet 45-70 cm snø i dagene etter mil...,62473,6916553,33,2018-01-03T00:00:00,2018-01-03T23:59:59
3,Halvor@NVE,Perioder med litt snø og vind de siste dagene ...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",Det er observert overflaterim flere steder i r...,2,2 Moderat,Ikke gitt,1,Det er ikke registrert skredaktivitet de siste...,...,3024,Sunnmøre,10,A,Det kom 45-70 cm snø i dagene etter mildværet ...,62473,6916553,33,2018-01-04T00:00:00,2018-01-04T23:59:59
4,Halvor@NVE,Perioder med litt snø og vind de siste dagene ...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",Det er observert overflaterim flere steder i r...,2,2 Moderat,Ikke gitt,1,Det er ikke registrert skredaktivitet de siste...,...,3024,Sunnmøre,10,A,23.12 var det mildvær og regn til topps. Deret...,62473,6916553,33,2018-01-05T00:00:00,2018-01-05T23:59:59


### List relevant parameters for the regression analysis

In [12]:
for dt,DL, APs in zip(df['ValidFrom'], df['DangerLevel'],df['AvalancheProblems']):
    for AP in APs:
        print(dt, DL, AP['AvalancheProblemTypeName'], AP['DestructiveSizeExtName'], AP['AvalTriggerSimpleName'], AP['AvalancheProblemId'])

2018-01-01T00:00:00 2 Fokksnø 2 - Små Stor tilleggsbelastning 1
2018-01-02T00:00:00 2 Fokksnø 2 - Små Stor tilleggsbelastning 1
2018-01-03T00:00:00 2 Fokksnø 2 - Små Liten tilleggsbelastning 1
2018-01-04T00:00:00 2 Vedvarende svakt lag 2 - Små Liten tilleggsbelastning 1
2018-01-05T00:00:00 2 Vedvarende svakt lag 2 - Små Liten tilleggsbelastning 1
2018-01-06T00:00:00 2 Vedvarende svakt lag 2 - Små Liten tilleggsbelastning 1
2018-01-06T00:00:00 2 Nysnøflak 2 - Små Liten tilleggsbelastning 2
2018-01-07T00:00:00 3 Vedvarende svakt lag 2 - Små Liten tilleggsbelastning 2
2018-01-07T00:00:00 3 Fokksnø 3 - Middels Naturlig utløst 1
2018-01-08T00:00:00 3 Fokksnø 3 - Middels Liten tilleggsbelastning 1
2018-01-08T00:00:00 3 Vedvarende svakt lag 2 - Små Liten tilleggsbelastning 2
2018-01-09T00:00:00 2 Fokksnø 2 - Små Stor tilleggsbelastning 1
2018-01-09T00:00:00 2 Vedvarende svakt lag 2 - Små Liten tilleggsbelastning 2
2018-01-10T00:00:00 2 Fokksnø 2 - Små Stor tilleggsbelastning 1
2018-01-10T00:0